In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_pv = pd.read_excel("/content/drive/Shareddrives/Deloitte Case Comp Spring 2021/Datathon_CY21/MD_PV_NetGen_20102020.xlsx", header = 1)
df_loc = pd.read_excel("/content/drive/Shareddrives/Deloitte Case Comp Spring 2021/Datathon_CY21/MD_PV_Locations.xlsx")

# Data Cleaning

In [ ]:
df_pv.head(5)

,Plant Id,Combined Heat And\nPower Plant,Plant Name,Operator Name,Operator Id,Plant State,NAICS Code,Sector Name,Reported\nPrime Mover,Tot_MMBtu\nJanuary,Tot_MMBtu\nFebruary,Tot_MMBtu\nMarch,Tot_MMBtu\nApril,Tot_MMBtu\nMay,Tot_MMBtu\nJune,Tot_MMBtu\nJuly,Tot_MMBtu\nAugust,Tot_MMBtu\nSeptember,Tot_MMBtu\nOctober,Tot_MMBtu\nNovember,Tot_MMBtu\nDecember,Elec_MMBtu\nJanuary,Elec_MMBtu\nFebruary,Elec_MMBtu\nMarch,Elec_MMBtu\nApril,Elec_MMBtu\nMay,Elec_MMBtu\nJune,Elec_MMBtu\nJuly,Elec_MMBtu\nAugust,Elec_MMBtu\nSeptember,Elec_MMBtu\nOctober,Elec_MMBtu\nNovember,Elec_MMBtu\nDecember,Netgen\nJanuary,Netgen\nFebruary,Netgen\nMarch,Netgen\nApril,Netgen\nMay,Netgen\nJune,Netgen\nJuly,Netgen\nAugust,Netgen\nSeptember,Netgen\nOctober,Netgen\nNovember,Netgen\nDecember,Total Fuel Consumption\nMMBtu,Elec Fuel Consumption\nMMBtu,Net Generation\n(Megawatthours),YEAR
0,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,9288,18006,14649,16100,18166,20107,20642,18193,16252,13260,11479,8531,9288,18006,14649,16100,18166,20107,20642,18193,16252,13260,11479,8531,1043,2022,1645,1808,2040,2258,2318,2043,1825,1489,1289,958.0,184673,184673,20738.0,2020
1,58408,N,Maryland Solar,"CD Arevon USA, Inc.",61230,MD,22,NAICS-22 Non-Cogen,PV,15219,15290,23376,26840,29769,34899,36199,30571,26893,21978,18398,11015,15219,15290,23376,26840,29769,34899,36199,30571,26893,21978,18398,11015,1709,1717,2625,3014,3343,3919,4065,3433,3020,2468,2066,1237.0,290447,290447,32616.0,2020
2,59851,N,Great Bay Solar 1,Great Bay Solar I LLC,59633,MD,22,NAICS-22 Non-Cogen,PV,71659,73137,98222,113245,125409,128811,126166,105275,95889,85399,80323,60892,71659,73137,98222,113245,125409,128811,126166,105275,95889,85399,80323,60892,8047,8213,11030,12717,14083,14465,14168,11822,10768,9590,9020,6838.0,1164427,1164427,130761.0,2020
3,60735,N,Longview Solar,GSRP,61944,MD,22,NAICS-22 Non-Cogen,PV,10793,12966,17659,17400,24231,28923,28051,20927,16617,13322,10731,8834,10793,12966,17659,17400,24231,28923,28051,20927,16617,13322,10731,8834,1212,1456,1983,1954,2721,3248,3150,2350,1866,1496,1205,992.0,210454,210454,23633.0,2020
4,60737,N,Church Hill,GSRP,61944,MD,22,NAICS-22 Non-Cogen,PV,4987,5619,6812,7249,8629,9413,9484,8326,6964,6109,5779,4265,4987,5619,6812,7249,8629,9413,9484,8326,6964,6109,5779,4265,560,631,765,814,969,1057,1065,935,782,686,649,479.0,83636,83636,9392.0,2020


In [ ]:
df_loc.head(5)

,Plant Name,Address,Zip Code,Lattitude,Longitude
0,Mount Saint Mary's,16300 Old Emmitsburg Road,21727,39.675000,-77.345000
1,Maryland Solar,18695 Roxbury Road,21746,39.563056,-77.720278
2,Great Bay Solar 1,Corner of Old Princess Anne Rd,21853,38.168889,-75.697500
3,Longview Solar,7340 Levin Dashiell Road,21830,38.394728,-75.672715
4,Church Hill,630 Cedar Lane,21623,39.100947,-75.959681


#### Edit column names of two dataframes

In [ ]:
colnames = df_pv.columns

df_pv.columns = colnames.str.replace("\n", "_").str.replace(" ", "_").str.strip().str.replace("(", "").str.replace(")","").str.title()

In [ ]:
df_pv.head(5)

,Plant_Id,Combined_Heat_And_Power_Plant,Plant_Name,Operator_Name,Operator_Id,Plant_State,Naics_Code,Sector_Name,Reported_Prime_Mover,Tot_Mmbtu_January,Tot_Mmbtu_February,Tot_Mmbtu_March,Tot_Mmbtu_April,Tot_Mmbtu_May,Tot_Mmbtu_June,Tot_Mmbtu_July,Tot_Mmbtu_August,Tot_Mmbtu_September,Tot_Mmbtu_October,Tot_Mmbtu_November,Tot_Mmbtu_December,Elec_Mmbtu_January,Elec_Mmbtu_February,Elec_Mmbtu_March,Elec_Mmbtu_April,Elec_Mmbtu_May,Elec_Mmbtu_June,Elec_Mmbtu_July,Elec_Mmbtu_August,Elec_Mmbtu_September,Elec_Mmbtu_October,Elec_Mmbtu_November,Elec_Mmbtu_December,Netgen_January,Netgen_February,Netgen_March,Netgen_April,Netgen_May,Netgen_June,Netgen_July,Netgen_August,Netgen_September,Netgen_October,Netgen_November,Netgen_December,Total_Fuel_Consumption_Mmbtu,Elec_Fuel_Consumption_Mmbtu,Net_Generation_Megawatthours,Year
0,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,9288,18006,14649,16100,18166,20107,20642,18193,16252,13260,11479,8531,9288,18006,14649,16100,18166,20107,20642,18193,16252,13260,11479,8531,1043,2022,1645,1808,2040,2258,2318,2043,1825,1489,1289,958.0,184673,184673,20738.0,2020
1,58408,N,Maryland Solar,"CD Arevon USA, Inc.",61230,MD,22,NAICS-22 Non-Cogen,PV,15219,15290,23376,26840,29769,34899,36199,30571,26893,21978,18398,11015,15219,15290,23376,26840,29769,34899,36199,30571,26893,21978,18398,11015,1709,1717,2625,3014,3343,3919,4065,3433,3020,2468,2066,1237.0,290447,290447,32616.0,2020
2,59851,N,Great Bay Solar 1,Great Bay Solar I LLC,59633,MD,22,NAICS-22 Non-Cogen,PV,71659,73137,98222,113245,125409,128811,126166,105275,95889,85399,80323,60892,71659,73137,98222,113245,125409,128811,126166,105275,95889,85399,80323,60892,8047,8213,11030,12717,14083,14465,14168,11822,10768,9590,9020,6838.0,1164427,1164427,130761.0,2020
3,60735,N,Longview Solar,GSRP,61944,MD,22,NAICS-22 Non-Cogen,PV,10793,12966,17659,17400,24231,28923,28051,20927,16617,13322,10731,8834,10793,12966,17659,17400,24231,28923,28051,20927,16617,13322,10731,8834,1212,1456,1983,1954,2721,3248,3150,2350,1866,1496,1205,992.0,210454,210454,23633.0,2020
4,60737,N,Church Hill,GSRP,61944,MD,22,NAICS-22 Non-Cogen,PV,4987,5619,6812,7249,8629,9413,9484,8326,6964,6109,5779,4265,4987,5619,6812,7249,8629,9413,9484,8326,6964,6109,5779,4265,560,631,765,814,969,1057,1065,935,782,686,649,479.0,83636,83636,9392.0,2020


In [ ]:
colnames = df_loc.columns
df_loc.columns = colnames.str.replace("\n", "_").str.replace(" ", "_").str.strip().str.replace("(", "").str.replace(")","").str.title()

In [ ]:
df_pv[df_pv['Year'] == 2010]

,Plant_Id,Combined_Heat_And_Power_Plant,Plant_Name,Operator_Name,Operator_Id,Plant_State,Naics_Code,Sector_Name,Reported_Prime_Mover,Tot_Mmbtu_January,Tot_Mmbtu_February,Tot_Mmbtu_March,Tot_Mmbtu_April,Tot_Mmbtu_May,Tot_Mmbtu_June,Tot_Mmbtu_July,Tot_Mmbtu_August,Tot_Mmbtu_September,Tot_Mmbtu_October,Tot_Mmbtu_November,Tot_Mmbtu_December,Elec_Mmbtu_January,Elec_Mmbtu_February,Elec_Mmbtu_March,Elec_Mmbtu_April,Elec_Mmbtu_May,Elec_Mmbtu_June,Elec_Mmbtu_July,Elec_Mmbtu_August,Elec_Mmbtu_September,Elec_Mmbtu_October,Elec_Mmbtu_November,Elec_Mmbtu_December,Netgen_January,Netgen_February,Netgen_March,Netgen_April,Netgen_May,Netgen_June,Netgen_July,Netgen_August,Netgen_September,Netgen_October,Netgen_November,Netgen_December,Total_Fuel_Consumption_Mmbtu,Elec_Fuel_Consumption_Mmbtu,Net_Generation_Megawatthours,Year
311,99999,N,State-Fuel Level Increment,State-Fuel Level Increment,99999,MD,99999,NAICS-22 Non-Cogen,PV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.354,784,784,80.354,2010


#### Change dataframe df_pv from unstacked to stacked (wide to long)

In [ ]:
mmbtu = df_pv.iloc[:,9:21].transpose(copy=True)

netgen = df_pv.iloc[:,33:45].transpose(copy=True)

In [ ]:
mmbtu

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311
Tot_Mmbtu_January,9288,15219,71659,10793,4987,1558,1247,4456,158537,8104,1323,1180,7703,874,2847,783,823,929,961,2777,12298,4640,1447,586,950,1725,11037,703,3290,71222,838,1517,1520,974,1851,802,1100,1334,1482,9805,...,594,698,727,704,1930,14531,3441,1098,515,.,654,2345,1054,1066,10752,732,.,651,749,843,820,.,.,4011,.,586,1448,1505,.,1051,0,.,.,.,.,.,1524,0,0,NaN
Tot_Mmbtu_February,18006,15290,73137,12966,5619,1621,1514,5293,190499,9723,1302,1161,10704,861,2803,771,810,915,947,2734,16065,4569,1425,577,935,1699,10866,692,3239,72896,825,1494,1497,959,1822,790,1083,1313,1459,9654,...,749,880,917,887,2432,18316,4337,1384,650,.,824,2551,1454,1470,14823,1010,1403,897,1033,1162,1130,2800,.,5530,.,808,1044,1085,.,758,0,645,.,.,.,.,1982,0,0,NaN
Tot_Mmbtu_March,14649,23376,98222,17659,6812,2306,2066,6407,234650,11444,2023,1804,15904,1337,4353,1198,1258,1421,1470,4246,26670,7095,2213,896,1452,2638,16875,1075,5031,109532,1281,2320,2325,1490,2830,1227,1681,2039,2266,14991,...,1125,1321,1377,1332,3654,27512,6514,2079,976,.,1238,3907,1879,1901,19167,1306,1815,1160,1336,1502,1461,3621,.,7151,.,1044,2399,2492,.,1741,0,1482,.,.,.,.,3360,0,0,NaN
Tot_Mmbtu_April,16100,26840,113245,17400,7249,2137,2395,7925,291685,14147,2125,1895,16519,1404,4573,1258,1322,1493,1544,4461,29306,7454,2325,941,1526,2771,17729,1130,5285,118659,1346,2437,2442,1565,2973,1289,1766,2142,2381,15750,...,1362,1600,1667,1613,4424,33314,7888,2517,1182,.,1499,4285,1613,1631,16446,1120,1557,995,1146,1289,1254,3107,.,6136,.,896,2701,2806,.,1960,0,1669,.,.,.,.,2229,0,0,NaN
Tot_Mmbtu_May,18166,29769,125409,24231,8629,3028,2814,9093,348811,16423,2292,2044,16020,1515,4933,1357,1426,1610,1666,4811,32218,8039,2508,1015,1646,2989,19121,1219,5700,132952,1451,2628,2634,1688,3207,1390,1905,2311,2568,16987,...,1614,1896,1976,1911,5242,39476,9347,2983,1400,.,1777,4896,2371,2398,24178,1647,2289,1463,1685,1895,1844,4568,.,9021,.,1317,2432,2527,.,1766,0,1503,.,.,.,.,2297,0,0,NaN
Tot_Mmbtu_June,20107,34899,128811,28923,9413,2948,3197,8886,339373,16017,2035,1815,20357,1345,4380,1205,1266,1430,1479,4272,35869,7139,2227,901,1461,2654,16979,1082,5062,132595,1289,2334,2339,1499,2847,1234,1692,2052,2280,15084,...,1728,2031,2116,2047,5615,42281,10011,3195,1500,.,1903,5245,2394,2421,24412,1663,2311,1478,1701,1913,1861,4612,.,9108,.,1330,2107,2189,.,1529,0,1302,.,.,.,.,1183,0,0,NaN
Tot_Mmbtu_July,20642,36199,126166,28051,9484,2805,3188,9619,366368,17292,2167,1932,22013,1432,4664,1283,1348,1522,1575,4549,37695,7602,2371,960,1556,2826,18080,1152,5390,136763,1372,2485,2491,1596,3032,1314,1801,2185,2428,16062,...,1629,1913,1994,1929,5290,39837,9433,3010,1413,.,1793,4811,2233,2259,22776,1552,2156,1379,1587,1785,1737,4303,.,8498,.,1241,2555,2654,27385,1854,0,1578,.,.,1909,.,1406,0,0,NaN
Tot_Mmbtu_August,18193,30571,105275,20927,8326,2600,2672,8358,321645,15114,1973,1759,19983,1304,4246,1168,1228,1386,1434,4142,35023,6921,2159,874,1417,2573,16461,1049,4907,108196,1250,2263,2268,1453,2760,1197,1640,1989,2211,14623,...,1570,1844,1922,1859,5100,38401,9093,2902,1362,.,1728,4781,2029,2052,20694,1410,1959,1253,1442,1622,1578,3909,.,7721,.,1128,1929,2004,20674,1400,0,1192,1493,.,1441,.,1273,0,0,NaN
Tot_Mmbtu_September,16252,26893,95889,16617,6964,2413,2244,6919,265308,12488,2020,1802,18006,1335,4348,1196,1257,1419,1468,4241,30615,7087,2211,895,1451,2635,16856,1074,5025,96869,1280,2317,2322,1488,2827,1225,1679,2037,2264,14975,...,1395,1639,1708,1652,4533,34131,8082,2579,1211,.,1536,4604,2120,2144,21618,1473,2047,1308,1507,1694,1648,4084,37763,8066,.,1178,1674,1739,17940,1215,0,1034,1295,.,1251,.,1964,1328,595,NaN
Tot_Mmbtu_October,13260,21978,85399,13322,6109,2013,1701,6464,248888,11596,1652,1473,12503,1092,3556,978,1028,1160,1201,3468,22503,5795,180

In [ ]:
names = list(df_pv.columns[:9])
names.append("Month_Year")
names.append("Mmbtu")
names.append("Netgen")
newdf = pd.DataFrame(columns = names)
newdf

,Plant_Id,Combined_Heat_And_Power_Plant,Plant_Name,Operator_Name,Operator_Id,Plant_State,Naics_Code,Sector_Name,Reported_Prime_Mover,Month_Year,Mmbtu,Netgen


In [ ]:
# y means month (1 to 12)
i = 0

for z in range(0, len(df_pv)):
      for y in range(1,13):
        row = list(df_pv.iloc[z,:9].values)
   #     x = df_pv.loc[z,"Year"]
        row.append(str(y) + "/" + str(df_pv.loc[z,'Year']))
        row.append(mmbtu.iloc[(y-1),z])
        row.append(netgen.iloc[(y-1),z])
        newdf.loc[i] = row
        i+=1

In [ ]:
newdf.head(10)

,Plant_Id,Combined_Heat_And_Power_Plant,Plant_Name,Operator_Name,Operator_Id,Plant_State,Naics_Code,Sector_Name,Reported_Prime_Mover,Month_Year,Mmbtu,Netgen
0,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,1/2020,9288,1043
1,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,2/2020,18006,2022
2,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,3/2020,14649,1645
3,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,4/2020,16100,1808
4,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,5/2020,18166,2040
5,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,6/2020,20107,2258
6,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,7/2020,20642,2318
7,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,8/2020,18193,2043
8,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,9/2020,16252,1825
9,57758,N,Mount Saint Mary's,Constellation Solar Horizons LLC,57065,MD,22,NAICS-22 Non-Cogen,PV,10/2020,13260,1489


In [ ]:
newdf.to_csv("/content/drive/Shareddrives/Deloitte Case Comp Spring 2021/Code/df_pv_cleaned.csv")